In [1]:
import numpy as np
import pandas as pd
import polars as pl
import torch

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'colab'

In [2]:
movies = pd.read_csv('data/autoencoders/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [3]:
users = pd.read_csv('data/autoencoders/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
users

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [4]:
ratings = pd.read_csv('data/autoencoders/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [24]:
train = pd.read_csv('data/autoencoders/u1.base', delimiter='\t')
train.columns = ['user', 'movie', 'rating', 'id']
train_arr = train.values
train

,user,movie,rating,id
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [25]:
test = pd.read_csv('data/autoencoders/u1.test', delimiter='\t')
test.columns = ['user', 'movie', 'rating', 'id']
test_arr = test.values
test

,user,movie,rating,id
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883
...,...,...,...,...
19994,458,648,4,886395899
19995,458,1101,4,886397931
19996,459,934,3,879563639
19997,460,10,3,882912371


In [26]:
n_users = max(train.user.max(), test.user.max())
n_movies = max(train.movie.max(), test.movie.max())

In [27]:
ratings_matrix = [[0 for _ in range(n_movies)] for _ in range(n_users)]
for i, r in train.iterrows():
    ratings_matrix[r.user-1][r.movie-1] = r.rating

In [30]:
# use df pivot
train.pivot(index='user', columns='movie', values='rating').fillna(0)

movie,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
def convert(data):
    ratings_matrix = [[0 for _ in range(n_movies)] for _ in range(n_users)]
    for i, r in data.iterrows():
        ratings_matrix[r.user-1][r.movie-1] = r.rating
    return ratings_matrix

train_conv = convert(train)
test_conv = convert(test)

In [36]:
train_tensor = torch.FloatTensor(train_conv)
test_tensor = torch.FloatTensor(test_conv)